# Загрузка Order book из CSV

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
from alex.database import Database
from pprint import pprint

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Скоростная загрузка с диска, из заархивированного CSV-файла

In [6]:
# результат выполнения запроса:
# select * from tvf_get_order_book_snapshots(2880) where exchange='binance' and pair='ETH/USDT' # запрос на 20 минут
order = pd.read_csv('tvf_get_order_book_snapshots 15.09.2018-01.10.2018.zip', index_col='dt')

In [3]:
# загрузка истории
# 15 секунд
db = Database()
sql_history = f"select * from v_history with (snapshot) where exchange='binance' and pair='ETH/USDT' and dt>'2018-09-23'"
df_history = db.query(sql_history)
df_history.set_index('dt', inplace=True)

### Немного статистики по загруженным датасетам

In [7]:
order.describe()

,exchange,pair,bid_ask,price,amount,volume
count,153400,153400,153400,153400,153400,153400
unique,1,1,2,1509,22817,29503
top,binance,ETH/USDT,ask,232,1,"10,06027"
freq,153400,153400,76700,682,3122,500


In [ ]:
len(order.index.sort_values().unique())

In [5]:
len(df_history)

1228706

### Получение Order Book в другом виде, напрямую из исторической таблицы dbo.order_book

Это не снимки, а только новые записи, добавляемые в order book в момент отрабатывания воркера. Поля ValidFrom и ValidTill показывают время жизни каждого ордера

In [23]:
# должно быть около 30 секунд
sql_orderbook = "SELECT *, ValidFrom, ValidTill FROM mem.order_book for system_time from '2018-09-29 23:23:00' to '2018-09-30 05:25:00' where id_ex_pair=19"
# id_ex_pair=19 - это Binance + ETH/USDT
df_orderbook = db.query(sql_orderbook)

In [24]:
df_orderbook['dt'] = pd.to_datetime(df_orderbook.dt)
df_orderbook['id'] = df_orderbook['id'].astype(int)
df_orderbook['ValidFrom'] = pd.to_datetime(df_orderbook.ValidFrom)
df_orderbook['ValidTill'] = pd.to_datetime(df_orderbook.ValidTill)
del df_orderbook['id_ex_pair']

In [25]:
df_orderbook.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66107 entries, 0 to 66106
Data columns (total 7 columns):
id           66107 non-null int32
dt           66107 non-null datetime64[ns]
is_bid       66107 non-null bool
price        66107 non-null float64
amount       66107 non-null float64
ValidFrom    66107 non-null datetime64[ns]
ValidTill    66107 non-null datetime64[ns]
dtypes: bool(1), datetime64[ns](3), float64(2), int32(1)
memory usage: 2.8 MB


In [26]:
# dt - время загрузки, id - порядковый номер записи, обычный инкремент identity(1,1)
df_orderbook.sort_values(['dt','id'], inplace=True)
df_orderbook.set_index(['dt','id'], inplace=True)

##### Скрипт загружает Order Book 1 раз в 3 секунды. Интервалы более 3 секунд между соседними записями могут означать, что Order Book не менялся, либо по каким-то причинам увеличилось время срабатывания скрипта

In [28]:
df_orderbook.tail(100)

is_bid   price     amount           ValidFrom  \
dt                  id                                                        
2018-09-30 05:24:22 32840542    True  227.47    0.57339 2018-09-30 05:24:23   
2018-09-30 05:24:25 32840685   False  233.05   70.56138 2018-09-30 05:24:26   
                    32840686   False  230.40   21.52500 2018-09-30 05:24:26   
                    32840687    True  230.11    1.18714 2018-09-30 05:24:26   
                    32840688   False  230.32   20.48352 2018-09-30 05:24:26   
                    32840689    True  230.01    1.00000 2018-09-30 05:24:26   
                    32840690    True  229.96   22.74600 2018-09-30 05:24:26   
                    32840691    True  229.77    3.50000 2018-09-30 05:24:26   
                    32840692   False  230.22    9.99254 2018-09-30 05:24:26   
2018-09-30 05:24:28 32840749    True  230.11    0.85834 2018-09-30 05:24:30   
                    32840750    True  230.10    1.75386 2018-09-30 05:24:30   
                    32840751   False  230.53    4.35187 2018-09-30 05:24:30   
                    32840752    True  230.12    0.58118 2018-09-30 05:24:30   
                    32840753    True  229.07   15.81587 2018-09-30 05:24:30   
                    32840754   False  232.91    0.08668 2018-09-30 05:24:30   
                    32840755   False  232.75    7.10276 2018-09-30 05:24:30   
                    32840756    True  230.01    1.32880 2018-09-30 05:24:30   
                    32840757   False  230.38   27.08756 2018-09-30 05:24:30   
2018-09-30 05:24:31 32840848   False  230.22   10.99254 2018-09-30 05:24:32   
                    32840849    True  229.02   11.15473 2018-09-30 05:24:32   
                    32840850    True  230.12    0.93130 2018-09-30 05:24:32   
                    32840851   False  230.32   19.48352 2018-09-30 05:24:32   
                    32840852    True  229.40    4.35187 2018-09-30 05:24:32   
                    32840853   False  230.86    6.00000 2018-09-30 05:24:32   
                    32840854    True  230.01    0.32880 2018-09-30 05:24:32   
                    32840855   False  233.08    0.08671 2018-09-30 05:24:32   
                    32840856    True  229.05    0.15000 2018-09-30 05:24:32   
2018-09-30 05:24:34 32840940    True  229.78    3.85050 2018-09-30 05:24:35   
                    32840941    True  229.59    0.16987 2018-09-30 05:24:35   
                    32840942   False  230.22    9.11424 2018-09-30 05:24:35   
...                              ...     ...        ...                 ...   
2018-09-30 05:24:46 32841310   False  230.20   11.92248 2018-09-30 05:24:47   
                    32841311    True  228.57    7.02175 2018-09-30 05:24:47   
                    32841312    True  228.81   10.90712 2018-09-30 05:24:47   
                    32841313   False  230.40   21.50600 2018-09-30 05:24:47   
                    32841314    True  228.01    5.46139 2018-09-30 05:24:47   
                    32841315    True  227.92    4.85900 2018-09-30 05:24:47   
                    32841316    True  230.11    1.32952 2018-09-30 05:24:47   
                    32841317    True  228.00  392.87057 2018-09-30 05:24:47   
                    32841318    True  227.81    0.51501 2018-09-30 05:24:47   
2018-09-30 05:24:49 32841481   False  230.19    6.03155 2018-09-30 05:24:51   
                    32841482    True  230.00   42.15385 2018-09-30 05:24:51   
                    32841483   False  230.20   10.95948 2018-09-30 05:24:51   
                    32841484    True  228.00  393.45819 2018-09-30 05:24:51   
                    32841485    True  229.88    4.70864 2018-09-30 05:24:51   
                    32841486    True  230.01    0.90000 2018-09-30 05:24:51   
                    32841487    True  230.07    1.00000 2018-09-30 05:24:51   
2018-09-30 05:24:52 32841556   False  230.17    0.20277 2018-09-30 05:24:53   
                    32841557    True  229.59    4.41987 2018-09-30 05:24:53   
 

In [69]:
df_orderbook.describe()

,id_ex_pair,price,amount
count,2317073.0,2.317073e+06,2.317073e+06
mean,19.0,2.254567e+02,2.116770e+01
std,0.0,1.035779e+01,8.315237e+01
min,19.0,2.016800e+02,1.000000e-05
25%,19.0,2.165800e+02,9.630000e-01
50%,19.0,2.259200e+02,4.000000e+00
75%,19.0,2.330500e+02,1.704812e+01
max,19.0,2.523100e+02,4.091701e+03


In [70]:
# кол-во записей в Order Book
len(df_orderbook)

2317073

In [76]:
# количество уникальных дат - дат загрузок Order Book
len(df_orderbook.index.unique(0))

225776